## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re
from collections import *
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [ ]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [ ]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [ ]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

Let's start by removing all parenthesized strings using a regex:

In [ ]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [ ]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [ ]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [ ]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [ ]:
len(sentences_ted)

In [ ]:
print(sentences_ted[0])
print(sentences_ted[1])

### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [ ]:
cnt = Counter()
for sentence in sentences_ted:
    for word in sentence:
        cnt[word] += 1
        
counts_ted_top1000 = []
for common_word in cnt.most_common(1000):
    counts_ted_top1000.append(common_word[1])

Plot distribution of top-1000 words

In [ ]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [ ]:
model_ted = Word2Vec(sentences_ted, min_count=10)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [ ]:
model_ted.most_similar("man")

In [ ]:
model_ted.most_similar("woman")

In [ ]:
model_ted.most_similar("dream")

In [ ]:
model_ted.most_similar("marriage")

In [ ]:
model_ted.most_similar("technology")

In [ ]:
model_ted.most_similar("entertainment")

In [ ]:
model_ted.most_similar("design")

In [ ]:
print(model_ted.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])[:3])
print(model_ted.most_similar_cosmul(positive=['computer', 'mind'], negative=['machine'])[:3])
print(model_ted.most_similar_cosmul(positive=['life','meaning'])[:3])
print(model_ted.most_similar_cosmul(positive=['war', 'usa'])[:3])

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [ ]:
words_top_ted = []
for common_word in cnt.most_common(1000):
    words_top_ted.append(common_word[0])

# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Extra: Literatura

In [ ]:
with open('/home/matheus/Documents/litearatura/copilados', 'r') as in_f:
    liter_text = in_f.read().replace('!','.').replace('?','.').lower().split('.')

liter_text[:3]

In [ ]:
for s_i in range(len(liter_text)):
    liter_text[s_i] = re.sub("[^a-z]", " ", liter_text[s_i]).split()

liter_text[:2]

In [ ]:
shuffle(liter_text)
print(len(liter_text))

In [ ]:
model_liter = Word2Vec(liter_text, min_count=5)

In [ ]:
model_liter.most_similar("homem")

In [ ]:
model_liter.most_similar("mulher")

In [ ]:
model_liter.most_similar("crime")

In [ ]:
model_liter.most_similar("sonho")

In [ ]:
model_liter.most_similar("amor")

In [ ]:
model_liter.most_similar("morte")

In [ ]:
print(model_liter.most_similar_cosmul(positive=['roma', 'franca'], negative=['italia']), '\n')
print(model_liter.most_similar_cosmul(positive=['barco'], negative=['navegar']), '\n')
print(model_liter.most_similar_cosmul(positive=['guerra', 'paz']), '\n')
print(model_liter.most_similar_cosmul(positive=['homem'], negative=['deus']), '\n')
print(model_liter.most_similar_cosmul(positive=['labirinto', 'xadrez']), '\n')

In [ ]:
cnt = Counter()
for sentence in liter_text:
    for word in sentence:
        cnt[word] += 1
        
counts_liter_top1000 = []
for common_word in cnt.most_common(1000):
    counts_liter_top1000.append(common_word[0])
    
words_top_vec_liter = model_liter[counts_liter_top1000]

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
words_top_liter_tsne = tsne.fit_transform(words_top_vec_liter)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_liter_tsne[:,0],
                                    x2=words_top_liter_tsne[:,1],
                                    names=counts_liter_top1000))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [5]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [6]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [7]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)

for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i]).split()
    
del input_text 

In [8]:
shuffle(sentences_wiki)
print(len(sentences_wiki))

# sample 1/5 of the data
# sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
# print(len(sentences_wiki))

4267112


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [9]:
cnt = Counter()
for sentence in sentences_wiki:
    for word in sentence:
        cnt[word] += 1
        
counts_wiki_top1000 = []
for common_word in cnt.most_common(1000):
    counts_wiki_top1000.append(common_word[1])

In [10]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [11]:
model_wiki = Word2Vec(sentences_wiki, min_count=10)

In [12]:
model_wiki.most_similar("man")

[('woman', 0.7764450907707214),
 ('person', 0.6642335653305054),
 ('girl', 0.6522268652915955),
 ('boy', 0.6507839560508728),
 ('soldier', 0.6333359479904175),
 ('dog', 0.5978959798812866),
 ('kid', 0.593299150466919),
 ('lad', 0.5791682004928589),
 ('policeman', 0.5700370669364929),
 ('creature', 0.5658121705055237)]

In [13]:
model_wiki.most_similar("woman")

[('girl', 0.7851673364639282),
 ('man', 0.7764450311660767),
 ('prostitute', 0.7507532238960266),
 ('child', 0.7333062291145325),
 ('person', 0.6935303211212158),
 ('lover', 0.6874986886978149),
 ('sailor', 0.6771126985549927),
 ('maid', 0.6676993370056152),
 ('teenager', 0.6642816662788391),
 ('stripper', 0.6536609530448914)]

In [14]:
model_wiki.most_similar("marriage")

[('divorce', 0.764147162437439),
 ('betrothal', 0.7234775424003601),
 ('marriages', 0.7179222106933594),
 ('marrying', 0.6747463345527649),
 ('husband', 0.6662060618400574),
 ('remarriage', 0.6587846875190735),
 ('wife', 0.6547430157661438),
 ('daughter', 0.624983012676239),
 ('relationship', 0.6248632073402405),
 ('marry', 0.6196048259735107)]

In [15]:
model_wiki.most_similar("love")

[('dreams', 0.6967384219169617),
 ('smile', 0.6486513018608093),
 ('yearning', 0.6401641964912415),
 ('romance', 0.6393997669219971),
 ('duet', 0.6384182572364807),
 ('kiss', 0.6368460059165955),
 ('passion', 0.6309168934822083),
 ('lust', 0.6176827549934387),
 ('dreaming', 0.6168950796127319),
 ('dream', 0.6127188205718994)]

In [16]:
print(model_wiki.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])[:3])
print(model_wiki.most_similar_cosmul(positive=['computer', 'mind'], negative=['machine'])[:3])
print(model_wiki.most_similar_cosmul(positive=['life','meaning'])[:3])
print(model_wiki.most_similar_cosmul(positive=['paris', 'japan'], negative=['france'])[:3])

[('empress', 0.9597502946853638), ('queen', 0.9431621432304382), ('monarch', 0.9399044513702393)]
[('backstory', 0.9932377934455872), ('personality', 0.9871536493301392), ('understanding', 0.9737261533737183)]
[('sense', 0.5207391977310181), ('greatness', 0.5186395645141602), ('identity', 0.5175397992134094)]
[('tokyo', 0.9551795125007629), ('osaka', 0.8998973369598389), ('shibuya', 0.8791349530220032)]


In [17]:
words_top_wiki = []
for common_word in cnt.most_common(1000):
    words_top_wiki.append(common_word[0])


#### t-SNE visualization

In [18]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [19]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)